# Lin Regr

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from itertools import combinations


In [2]:
df = pd.read_csv('data_cleaned.csv')

## Filtering Out Not Interesting Data

In [3]:
df_for_corr = df.iloc[:,6:]

In [4]:
df_for_corr = df_for_corr.corr()

In [5]:
# checking positive values first

In [6]:
df_for_corr['poor_mental_health_days'].sort_values(ascending = False).head(20)

poor_mental_health_days               1.000000
frequent_mental_distress              0.953254
poor_physical_health_days             0.919823
frequent_physical_distress            0.878407
poor_health                           0.743129
smokers                               0.739957
insufficient_sleep                    0.691513
diabetes                              0.655154
premature_age_adjusted_mortality      0.647103
children_in_oiverty                   0.638065
food_insecurity                       0.608685
YPLL                                  0.591831
physically_inactive                   0.517016
children_in_singleparent_household    0.497505
teen_birth_rate                       0.495828
children_elig_lunch                   0.494406
LBW                                   0.472024
unemployment                          0.468792
income_inequality                     0.410123
obese                                 0.394903
Name: poor_mental_health_days, dtype: float64

In [7]:
columns_to_exclude = []

In [8]:
columns_to_exclude.extend(
    ['frequent_mental_distress', 'poor_physical_health_days', 'frequent_physical_distress', 
    'poor_health'] 
)

In [9]:
df_for_corr['poor_mental_health_days'].sort_values(ascending = True).head(20)

life_expectancy                   -0.633379
excessive_drinking                -0.620358
median_household_income           -0.559596
college                           -0.546793
food_environment_index            -0.403171
mammography_screenings            -0.342726
social_associations               -0.330864
exercise_acces                    -0.215343
asian                             -0.197639
uninsured_children                -0.183329
PCP                               -0.168898
dentists                          -0.154767
nonhispanic_whit                  -0.139804
hispanic                          -0.123743
homeownership                     -0.123582
high_school_graduation            -0.120535
notproficient_english             -0.110233
flu_vaccinated                    -0.094487
alcohol_impaired_driving_deaths   -0.078282
population                        -0.065633
Name: poor_mental_health_days, dtype: float64

In [10]:
columns_to_exclude.extend(
    ['life_expectancy'] 
)

In [11]:
df.drop(labels = columns_to_exclude, axis = 1, inplace = True)

In [12]:
df.shape

(3142, 63)

## Train - Test Split

In [13]:
# splitting the data into train and test

In [14]:
y = df['poor_mental_health_days']
X = df.drop(['FIPS_state', 'FIPS_county', 'FIPS_full', 'state', 'county', 'year',
       'YPLL', 'poor_mental_health_days'], axis = 1)

In [15]:
# check if we need to log / sqrt transform anything

In [16]:
# for column in X.columns: 
#     sns.scatterplot(X[column], y)
#     plt.show()

For now, none of these seem to be obviously log / sqrt types. They are either linear, or look independent. 

In [23]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

## Baseline Model

In [25]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Using a K-fold model overall, our dataset is quite small. 

In [26]:
regression = LinearRegression()

crossvalidation = KFold(n_splits=5, shuffle=True, random_state= 42)
baseline = np.mean(cross_val_score(regression, X_train_scaled, y_train, scoring='r2', cv=crossvalidation))
baseline

0.830323591643683

In [57]:
cross_val_score(regression, X_train_scaled, y_train, scoring='r2', cv=crossvalidation)

array([0.83576378, 0.83895288, 0.84466206, 0.80828791, 0.82395133])

In [58]:
cross_val_score(regression, X_train, y_train, scoring='r2', cv=crossvalidation)

array([0.83576378, 0.83895288, 0.84466206, 0.80828791, 0.82395133])

The baseline model's R^2 is 0.83, this is what we are going to compare against. 

## Interactions

In [72]:
combs = list(combinations(X_train.columns, 2))

In [73]:
# the cell runs for a while
interactions = []

for comb in combs:
    
    data = X_train.copy()
    
    data['interaction'] = data[comb[0]] * data[comb[1]]
    score = np.mean(cross_val_score(regression, data, y_train, scoring='r2', cv=crossvalidation))
    if score > baseline: interactions.append((comb[0], comb[1], round(score, 3)))
            

In [74]:
# we are adding the interactions that increased the average R_squared to over 0.835 (rounded to 3 decimal places)

In [75]:
interactions_to_add = [interaction for interaction in interactions if interaction[2] > 0.835]

In [76]:
interactions_to_add

[('obese', 'college', 0.836),
 ('uninsured', 'nonhisp_africanamerican', 0.836),
 ('air_pollution', 'insufficient_sleep', 0.836),
 ('severe_housing_problems', 'insufficient_sleep', 0.838),
 ('insufficient_sleep', 'severe_housing_cost_burden', 0.839),
 ('uninsured_children', 'nonhisp_africanamerican', 0.84)]

In [77]:
X_train_inter = X_train.copy()

In [78]:
for interaction in interactions_to_add:
    X_train_inter[interaction[0] + '_' + interaction[1]] = \
    X_train_inter[interaction[0]] * X_train_inter[interaction[1]]

In [79]:
interaction_score = np.mean(cross_val_score(regression, X_train_inter, y_train, scoring='r2', cv=crossvalidation))
interaction_score

0.8550484166399034

__Summary__: 
<br>

With the interactions in the <code>interactions_to_add</code> variable added to the morel, R<sup>2</sup> increased to 0.855. 

## Polynomials

In [96]:
polynomials = []

for column in X_train.columns:
    
    for degree in [2, 3]:

        data = X_train.copy()
        poly = PolynomialFeatures(degree, include_bias=False)
        X_transformed = poly.fit_transform(X_train[[column]])
        
        data.reset_index(drop=True, inplace=True)
        X_transformed = pd.DataFrame(X_transformed)
        X_transformed.reset_index(drop=True, inplace=True)
        
        data = pd.concat([data.drop(column, axis=1),X_transformed], axis=1)
        
        score = np.mean(cross_val_score(regression, data, y_train, scoring='r2', cv=crossvalidation))
        if score > baseline: polynomials.append((column, degree, round(score, 3)))


In [97]:
polynomials_to_add = [polynomial for polynomial in polynomials if polynomial[2] > 0.835]

In [98]:
polynomials_to_add

[('insufficient_sleep', 2, 0.84),
 ('insufficient_sleep', 3, 0.84),
 ('uninsured_children', 2, 0.837),
 ('uninsured_children', 3, 0.837)]